In [65]:
from django.conf import settings
import os
import tarfile
import uuid
import shutil
from tardis.tardis_portal.models import StorageBox, DataFileObject, DataFile
from tardis.tardis_portal.models.dataset import Dataset

In [2]:
#TODOsteve report bug with verified (unverified if one storage box doesn't work which has weird side effects like image loading and downloading)

{'default': {'ENGINE': 'django.db.backends.sqlite3',
  'HOST': '',
  'NAME': 'db.sqlite3',
  'PASSWORD': '',
  'PORT': '',
  'USER': ''}}

In [4]:
boxes = StorageBox.objects.all()

In [5]:
boxes

[<StorageBox: local box at /home/ubuntu/mytardis/var/store>, <StorageBox: tararchive box at /home/ubuntu/mytardis/var/tararchive>, <StorageBox: tararchive cache box at /home/ubuntu/mytardis/var/tararchive_cache>]

In [6]:
boxes[0]

<StorageBox: local box at /home/ubuntu/mytardis/var/store>

In [7]:
default_box = boxes[0]

In [8]:
# copy to tararchive box (won't do this in future, preferring cache)
# default_box.copy_files(dest_box=boxes[1])

In [9]:
cache_box = boxes[2]

In [55]:
print "dataset_id value in cache box"
dataset_id = cache_box.options.filter(key='dataset_id')[0].value
print dataset_id

dataset_id value in cache box
1


In [25]:
dataset = Dataset.objects.filter(id=dataset_id)[0]

In [12]:
# df = dataset.datafile_set.all()[0]
# dfo = df.file_objects.first()
# dfo._get_identifier()

In [13]:
# TODO: make <dataset_id>.tar in cache box

In [86]:
# task that goes through and creates tars where applicable
# boxes for dataset
# TODOsteve change 'tar_path' to 'location' option name

default_box = StorageBox.objects.filter(django_storage_class='tardis.tardis_portal.storage.MyTardisLocalFileSystemStorage').first()
# inferred cache path from tar box
dataset_default_path = default_box.options.filter(key='location').first().value
print 'default location: ' + dataset_default_path

tarcache_box = StorageBox.objects.filter(django_storage_class='tardis.tardis_portal.storage.tararchivefs.TarArchiveFileSystemStorageCache').first()
# inferred cache path from tar box
dataset_cache_path = tarcache_box.options.filter(key='tar_path').first().value
print 'cache location: ' + dataset_cache_path

boxes = StorageBox.objects.filter(options__key='dataset_id', django_storage_class='tardis.tardis_portal.storage.tararchivefs.TarArchiveFileSystemStorage')

for tararchive_box in boxes:
    
    dataset_id = tararchive_box.options.filter(key='dataset_id').first().value
    print 'dataset id: ' + dataset_id
    
    # corresponding tar box
    tararchive_box = StorageBox.objects.filter(options__key='dataset_id', options__value='1', django_storage_class='tardis.tardis_portal.storage.tararchivefs.TarArchiveFileSystemStorage').first()
    print 'tararchive box for dataset: ' + str(tararchive_box)
    
    dataset = dataset = Dataset.objects.filter(id=dataset_id).first()
    print 'dataset: ' + str(dataset)
    
    # inferred path from tar box
    dataset_tar_dir = tararchive_box.options.filter(key='tar_path').first().value
    dataset_tar = dataset_tar_dir + '/' + str(dataset.id) + '.tar'
    print dataset_tar
    
    # tar exists or not
    exists = os.path.isfile(dataset_tar)
    print exists
    
    # file list for tar
    for df in dataset.datafile_set.all():
        dfo = df.file_objects.first()
        print '\t' + dfo._storage.path(dfo.uri)


    unique_filename = uuid.uuid4()
    tmp_cache_tar = dataset_cache_path + '/' + str(unique_filename) + '.tar'
    print tmp_cache_tar
    
    # if tar archive doesn't exist
    # TODOsteve modified times and update/replace tar
    if not exists:
        # create tar in cache
        with tarfile.open(tmp_cache_tar, "w") as tar:
            for df in dataset.datafile_set.all():

                dfo = df.file_objects.first()
                file_path = dfo._storage.path(dfo.uri)
                
                info = tarfile.TarInfo(name=dfo.uri)
                info.size=os.path.getsize(file_path)
                
                tar.addfile(info,
                        file(file_path))
                
        # copy file to tar archive and rename to dataset_id.tar
        new_tar_name = str(dataset.id) + '.tar'
        print 'copying temp tar file ' + tmp_cache_tar + ' to ' + dataset_tar_dir
        shutil.copy2(tmp_cache_tar, dataset_tar_dir)
        os.rename(dataset_tar_dir + '/' + str(unique_filename) + '.tar',
                 dataset_tar)
        
        # remove tar from cache
        #os.remove(tmp_cache_tar)
        
        for df in dataset.datafile_set.all():
            dfo = df.file_objects.first()           
            
            # create dfo
            # TODOsteve create storage box in process
            copy = DataFileObject(
                datafile=df,
                storage_box=tararchive_box,
                uri=dfo.uri)
            copy.save()
            
            print 'creating dfo copy ' + str(df) + ' in ' + str(tararchive_box)
            
    # TODOsteve finally clean up on fail/success


default location: /home/ubuntu/mytardis/var/store
cache location: /home/ubuntu/mytardis/var/tararchive_cache
dataset id: 1
tararchive box for dataset: tararchive box at /home/ubuntu/mytardis/var/tararchive
dataset: A dataset
/home/ubuntu/mytardis/var/tararchive/1.tar
False
	/home/ubuntu/mytardis/var/store/A dataset-1/Photo 4-10-12 2 15 00 PM.png
	/home/ubuntu/mytardis/var/store/A dataset-1/SekZx.jpg
/home/ubuntu/mytardis/var/tararchive_cache/e9803a02-02c1-435c-a61c-9e8ffa2c1fa9.tar
copying temp tar file /home/ubuntu/mytardis/var/tararchive_cache/e9803a02-02c1-435c-a61c-9e8ffa2c1fa9.tar to /home/ubuntu/mytardis/var/tararchive
creating dfo copy 5905a2d2b357935196d4a81844c9bc65 Photo 4-10-12 2 15 00 PM.png # image/png in tararchive box at /home/ubuntu/mytardis/var/tararchive
creating dfo copy e3d70bc8d7173fc22b87f3db15bfe9d4 SekZx.jpg # image/jpeg in tararchive box at /home/ubuntu/mytardis/var/tararchive


In [85]:
# tararchive removal
dfos = DataFileObject.objects.filter(storage_box=tararchive_box)

for dfo in dfos:
    print dfo
    #dfo.delete()

In [68]:
# ALL FILE REMOVAL
# dfos = DataFileObject.objects.all()

# for dfo in dfos:
#    print dfo
#    dfo.delete()
    
# FILE LINKS TOO
# dfs = DataFile.objects.all()
# for df in dfs:
#    print df
#    df.delete()

In [111]:
import tarfile,os

#mkdir -p functionality
def ensure_dir(f):
    d = os.path.dirname(f)
    if not os.path.exists(d):
        os.makedirs(d)

os.chdir("/home/ubuntu/mytardis/var/tararchive")
tar = tarfile.open("1.tar")
#for member in tar.getmembers():
#    print member
#     f=tar.extractfile(member)
#     content=f.read()
#     print "%s has %d newlines" %(member, content.count("\n"))
#     print "%s has %d spaces" % (member,content.count(" "))
#     print "%s has %d characters" % (member, len(content))
#     sys.exit()

#filename = 'A dataset-1/Photo 4-10-12 2 15 00 PM.png'
filename = 'A dataset-1/SekZx.jpg'

write_path = '/home/ubuntu/mytardis/var/tararchive_cache/' + filename

member = tar.getmember(filename)

f=tar.extractfile(member)

ensure_dir(write_path)

with open(write_path, 'w+') as extracted_file:
    extracted_file.write(f.read())

tar.close()

'/home/ubuntu/mytardis/var/tararchive/A dataset-1'